In [1]:
import pandas as pd
import altair as alt

In [17]:
df = pd.read_csv("us_inflow_data.csv")
df.head(10)

,Year,Annual Ceiling,Number of Admitted Refugees
0,1975,-,"146,158"
1,1976,-,"27,206"
2,1977,-,"19,946"
3,1978,-,"36,507"
4,1979,-,"111,363"
5,1980,"231,700","207,116"
6,1981,"217,000","159,252"
7,1982,"140,000","98,096"
8,1983,"90,000","61,218"
9,1984,"72,000","70,393"


In [18]:

df = df[df["Annual Ceiling"] != "-"]

In [19]:
df["Annual Ceiling"] = df["Annual Ceiling"].str.replace(",", "").astype(int)
df['Year'] = pd.to_datetime(df['Year'], format='%Y')
df["Number of Admitted Refugees"] = df["Number of Admitted Refugees"].str.replace(",", "").astype(int)
df.head()

,Year,Annual Ceiling,Number of Admitted Refugees
5,1980-01-01,231700,207116
6,1981-01-01,217000,159252
7,1982-01-01,140000,98096
8,1983-01-01,90000,61218
9,1984-01-01,72000,70393


In [20]:
df.shape

(45, 3)

In [24]:
horizontal_brush = alt.selection_interval(encodings=["x"])
opacity_condition = alt.condition(horizontal_brush, alt.value(1), alt.value(0.2))

line_plot = (
    alt.Chart(df)
    .mark_line(color="orange", point=alt.MarkConfig(color="orange"))
    .encode(
        x=alt.X("Year:T"),
        y=alt.Y("Number of Admitted Refugees:Q", title="Number of Refugees"),
        opacity=opacity_condition,
    )
    .add_params(horizontal_brush)
    .properties(width=600, height=300, title="US Admissions vs Annual Ceiling")
)

line_plot_2 = (
    alt.Chart(df)
    .mark_line(color="blue", point=alt.MarkConfig(color="blue"))
    .encode(
        x=alt.X("Year:T"),
        y=alt.Y("Annual Ceiling:Q", title="Annual Ceiling"),
        opacity=opacity_condition,
    )
    .add_params(horizontal_brush)
    .properties(width=600, height=300, title="US Admissions vs Annual Ceiling")
)

total_admissions = (
    alt.Chart(df)
    .transform_filter(horizontal_brush)
    .transform_aggregate(total="sum(Number of Admitted Refugees)")
    .transform_calculate(text="Total Admissions: " + alt.datum.total)
    .mark_text(align="left", dx=-150, fontSize=20)
    .encode(text="text:N")
    .properties(width=600, height=50)
)

combined_chart = line_plot + line_plot_2 & total_admissions
combined_chart

alt.VConcatChart(...)